In [1]:
import jax.numpy as jnp

In [2]:
import numpyro

In [3]:
from lightweight_mmm import lightweight_mmm
from lightweight_mmm import optimize_media
from lightweight_mmm import plot
from lightweight_mmm import preprocessing
from lightweight_mmm import utils

In [16]:
import datetime
import numpy as np
import pandas as pd

In [6]:
df = pd.read_csv('source/Sample Media Spend Data.csv')
df.head()

,Division,Calendar_Week,Paid_Views,Organic_Views,Google_Impressions,Email_Impressions,Facebook_Impressions,Affiliate_Impressions,Overall_Views,Sales
0,A,1/6/2018,392,422,408,349895.0107,73580,12072,682,59417
1,A,1/13/2018,787,904,110,506270.2176,11804,9499,853,56806
2,A,1/20/2018,81,970,742,430042.1538,52232,17048,759,48715
3,A,1/27/2018,25,575,65,417745.6658,78640,10207,942,72047
4,A,2/3/2018,565,284,295,408505.8012,40561,5834,658,56235


In [15]:
df.dtypes

Division                  object
Calendar_Week             object
Paid_Views                 int64
Organic_Views              int64
Google_Impressions         int64
Email_Impressions        float64
Facebook_Impressions       int64
Affiliate_Impressions      int64
Overall_Views              int64
Sales                      int64
dtype: object

In [21]:
df['Calendar_Week'] = pd.to_datetime(df['Calendar_Week'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')

In [23]:
print(f"{len(df)} rows")
print(f"Unique divisions: {np.unique(df['Division'])}")
print(f"Minimum date week: {np.min(df['Calendar_Week'])}")
print(f"Maximum date week: {np.max(df['Calendar_Week'])}")

3051 rows
Unique divisions: ['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R'
 'S' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z']
Minimum date week: 2018-01-06
Maximum date week: 2020-02-29


In [24]:
df.describe()

,Paid_Views,Organic_Views,Google_Impressions,Email_Impressions,Facebook_Impressions,Affiliate_Impressions,Overall_Views,Sales
count,3051.000000,3051.000000,3.051000e+03,3.051000e+03,3.051000e+03,3051.000000,3051.000000,3.051000e+03
mean,15093.941658,13355.673222,8.861738e+05,7.605094e+05,2.691269e+05,22910.902655,27980.914127,1.859014e+05
std,30785.884979,24079.399689,1.355076e+06,6.260141e+05,4.665117e+05,21617.637497,52054.976687,2.322079e+05
min,1.000000,1.000000,7.000000e+00,4.089445e+04,2.900000e+01,910.000000,2.000000,1.543600e+04
25%,537.000000,712.500000,1.698280e+05,3.784969e+05,5.707350e+04,9127.000000,747.000000,7.339350e+04
50%,2699.000000,4110.000000,4.905310e+05,5.909708e+05,1.275230e+05,16658.000000,7879.000000,1.135730e+05
75%,17358.000000,16230.500000,1.022622e+06,9.622466e+05,2.835050e+05,27486.500000,34111.500000,2.029755e+05
max,518190.000000,270453.000000,1.715044e+07,7.317730e+06,7.558435e+06,175791.000000,635057.000000,3.575430e+06
